In [1]:
import gc
from sklearn.decomposition import PCA
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold,KFold
import os
import pickle
from sklearn.metrics import precision_score
from catboost import CatBoostClassifier
import dgl
from sklearn.model_selection import StratifiedKFold
import math
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
from copy import deepcopy
import torch

Using backend: pytorch


In [2]:
def load_dgl_graph_k_fold(base_path, fold=-1, k=10, seed=1996):

    with open(os.path.join(base_path, 'labels.pkl'), 'rb') as f:
        label_data = pickle.load(f)
    labels = torch.from_numpy(label_data['label'])
    labels = labels.to(torch.int64)
    test_label_idx = label_data['test_label_idx']
    if fold == -1:
        tr_label_idx = label_data['tr_label_idx']
        val_label_idx = label_data['val_label_idx']
    else:
        train_idx = np.concatenate((label_data['tr_label_idx'], label_data['val_label_idx']))
        folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
        for i, (tr, val) in enumerate(folds.split(train_idx, labels[train_idx])):
            tr_label_idx, val_label_idx = train_idx[tr], train_idx[val]
            if i == fold:
                print('    ###      use      fold: {}'.format(fold))
                break
    # get node features
    features = np.load(os.path.join(base_path, 'features.npy'))
    node_feat = torch.from_numpy(features).float()
    print('################ Feature info: ###############')
    print('Node\'s feature shape:{}'.format(node_feat.shape))
    return labels, tr_label_idx, val_label_idx, test_label_idx, node_feat

In [3]:
base_path = 'DGL'
graphs, _ = dgl.load_graphs(os.path.join(base_path, 'graph.bin'))
graph = graphs[0]
print('################ Graph info: ###############')
print(graph)

with open(os.path.join(base_path, 'labels.pkl'), 'rb') as f:
    label_data = pickle.load(f)

labels = torch.from_numpy(label_data['label'])
labels = labels.to(torch.int64)
tr_label_idx = label_data['tr_label_idx']
val_label_idx = label_data['val_label_idx']
test_label_idx = label_data['test_label_idx']
features = np.load(os.path.join(base_path, 'features.npy'))
features = torch.from_numpy(features).float()
print("load my features", features.shape)

################ Graph info: ###############
Graph(num_nodes=3655452, num_edges=29168650,
      ndata_schemes={}
      edata_schemes={})
load my features torch.Size([3655452, 300])


In [14]:
# 标签计数
labels
label_count = []
for i in range(23):
    nums = (labels == i).sum().numpy().tolist()
    label_count.append(nums)
# print(label_count/sum(label_count))
label_weight = [round(sum(label_count)/i/10) for i in label_count]
label_weight = torch.tensor(label_weight).float().to('cuda:0')

In [6]:
degrees = (graph.in_degrees() + graph.out_degrees()).numpy()
iso_set = set(np.where(degrees==0)[0])
label_nodes = np.array(sorted(list(set(tr_label_idx)|set(val_label_idx))))
valid_nodes = np.array(sorted(list(set(label_nodes) & iso_set)))
train_nodes = np.array(sorted(list(set(label_nodes) - iso_set)))

In [7]:
class FastTensorDataLoader:
    def __init__(self, *tensors, batch_size=32, shuffle=False):
        assert all(t.shape[0] == tensors[0].shape[0] for t in tensors)
        self.tensors = tensors

        self.dataset_len = self.tensors[0].shape[0]
        self.batch_size = batch_size
        self.shuffle = shuffle

        # Calculate # batches
        n_batches, remainder = divmod(self.dataset_len, self.batch_size)
        if remainder > 0:
            n_batches += 1
        self.n_batches = n_batches
    def __iter__(self):
        if self.shuffle:
            r = torch.randperm(self.dataset_len)
            self.tensors = [t[r] for t in self.tensors]
        self.i = 0
        return self

    def __next__(self):
        if self.i >= self.dataset_len:
            raise StopIteration
        batch = tuple(t[self.i:self.i+self.batch_size] for t in self.tensors)
        self.i += self.batch_size
        return batch

    def __len__(self):
        return self.n_batches


# batch_size = 4096
# train_data_loader = FastTensorDataLoader(
#     features[tr_label_idx],
#     labels[tr_label_idx],
#     batch_size=batch_size, shuffle=True)
# valid_data_loader = FastTensorDataLoader(
#     features[val_label_idx],
#     labels[val_label_idx],
#     batch_size=batch_size, shuffle=True)

In [15]:

epsilon = 1 - math.log(2)
def custom_loss_function(x, labels, weight = label_weight):
    y = F.cross_entropy(x, labels,weight = weight, reduction="none")
    y = torch.log(epsilon + y) - math.log(epsilon)
    return torch.mean(y)
loss_fn = nn.CrossEntropyLoss().to('cuda:0')
class ISO_Node_NN(nn.Module):  

    def __init__(self): 
        super(ISO_Node_NN, self).__init__()        
        self.net = nn.Sequential(
            nn.Linear(300,2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(2048,1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024,512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512,256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 23),
        )
        

    def forward(self, x):  
        x = self.net(x)
        return x


In [36]:
def adjust_learning_rate(optimizer, epoch):
    for param_group in optimizer.param_groups:
        param_group["lr"] = 0.001 + (epoch % 5)*0.001

n_epochs = 100
device = 'cuda:0'
model = ISO_Node_NN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=0)
print('# model parameters:', sum(param.numel() for param in model.parameters()))


loss_tra = []
loss_val = []
val_best = 1e9
stop_count = 0
for epoch in range(n_epochs):
    adjust_learning_rate(optimizer, epoch)
    model.train()
    epoch_loss = []
    c = 0
    print("Epoch: {}/{}".format(epoch, n_epochs))
    val_acc_list = []
    for X_sequence, target in train_data_loader:
        X_sequence, target = X_sequence.to(device), target.to(device)

        y_hat = model(X_sequence)
        train_loss = custom_loss_function(y_hat, target)
        
        val_batch_pred = torch.sum(torch.argmax(y_hat, dim=1) == target) / torch.tensor(target.shape[0])
        val_acc_list.append(val_batch_pred.detach().cpu().numpy())

        train_loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        epoch_loss.append(train_loss.item())
    loss_tra = np.mean(epoch_loss)
    print('loss_tra',loss_tra)
    print('acc train',np.mean(val_acc_list))

    # 验证集
    model.eval()
    epoch_loss = []
    val_acc_list = []
    for X_sequence, target in valid_data_loader:
        X_sequence, target = X_sequence.to(device), target.to(device)
        y_hat = model(X_sequence)
        valid_loss = custom_loss_function(y_hat, target)
        epoch_loss.append(valid_loss.item())

        val_batch_pred = torch.sum(torch.argmax(y_hat, dim=1) == target) / torch.tensor(target.shape[0])
        val_acc_list.append(val_batch_pred.detach().cpu().numpy())


    loss_val = np.mean(epoch_loss)
    print('loss_val',loss_val)
    print('acc val',np.mean(val_acc_list))
    if loss_val < val_best:
        val_best = loss_val
        acc_best = np.mean(val_acc_list)
        stop_count = 0
        best_model = deepcopy(model)
    else:
        stop_count += 1
    if stop_count == 15:
        break

# model parameters: 3421463
Epoch: 0/100
loss_tra 2.052369973970496
acc train 0.27717224
loss_val 1.818557124871474
acc val 0.36816004
Epoch: 1/100
loss_tra 1.8476709640544393
acc train 0.36485133
loss_val 1.7328547239303589
acc val 0.39682376
Epoch: 2/100
loss_tra 1.794464930244114
acc train 0.38836566
loss_val 1.6824805644842296
acc val 0.4244952
Epoch: 3/100
loss_tra 1.7593507694161457
acc train 0.40759435
loss_val 1.6593025510127728
acc val 0.43765703
Epoch: 4/100
loss_tra 1.7343575757482776
acc train 0.4184612
loss_val 1.6238491947834308
acc val 0.4496862
Epoch: 5/100
loss_tra 1.6932707174964574
acc train 0.43541712
loss_val 1.6022444184009845
acc val 0.4597008
Epoch: 6/100
loss_tra 1.6813772854597673
acc train 0.4403082
loss_val 1.596379422224485
acc val 0.46213436
Epoch: 7/100
loss_tra 1.6789348301680191
acc train 0.44102427
loss_val 1.5952856586529658
acc val 0.46303555
Epoch: 8/100
loss_tra 1.6803119773450106
acc train 0.44088903
loss_val 1.596023371586433
acc val 0.46177667
E

loss_val 1.5355803278776317
acc val 0.48958826
Epoch: 76/100
loss_tra 1.4862859015879424
acc train 0.51753914
loss_val 1.5322434122745807
acc val 0.49026197
Epoch: 77/100
loss_tra 1.4867913075115369
acc train 0.51700896
loss_val 1.5337762924341054
acc val 0.48923144
Epoch: 78/100
loss_tra 1.4958724104839822
acc train 0.51378065
loss_val 1.534531859251169
acc val 0.48904327
Epoch: 79/100
loss_tra 1.504378642724908
acc train 0.51064825
loss_val 1.5398644759104803
acc val 0.48756167
Epoch: 80/100
loss_tra 1.4886443656423818
acc train 0.5165693
loss_val 1.5326617772762592
acc val 0.4899357
Epoch: 81/100
loss_tra 1.4803277689477672
acc train 0.5196534
loss_val 1.5328511916674101
acc val 0.4905736
Epoch: 82/100
loss_tra 1.4824963030607805
acc train 0.5188248
loss_val 1.5289457577925463
acc val 0.49155837
Epoch: 83/100
loss_tra 1.4901759790337603
acc train 0.5158355
loss_val 1.534073068545415
acc val 0.48893884
Epoch: 84/100
loss_tra 1.5000636520593063
acc train 0.512388
loss_val 1.5384204754

In [18]:
base_path = 'DGL'
def adjust_learning_rate(optimizer, epoch):
    for param_group in optimizer.param_groups:
        param_group["lr"] = 0.001 + (epoch % 5)*0.001

n_epochs = 1000
device = 'cuda:0'

for fold in range(10):
    labels, tr_label_idx, val_label_idx, test_label_idx, node_feat = load_dgl_graph_k_fold(base_path,fold)
    batch_size = 4096
    train_data_loader = FastTensorDataLoader(
        node_feat[tr_label_idx],
        labels[tr_label_idx],
        batch_size=batch_size, shuffle=True)
    valid_data_loader = FastTensorDataLoader(
        node_feat[val_label_idx],
        labels[val_label_idx],
        batch_size=batch_size, shuffle=True)
    
    model = ISO_Node_NN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=0)
    print('# model parameters:', sum(param.numel() for param in model.parameters()))
    
    loss_tra = []
    loss_val = []
    val_best = 1e9
    stop_count = 0
    for epoch in range(n_epochs):
        adjust_learning_rate(optimizer, epoch)
        model.train()
        epoch_loss = []
        
        tra_acc_list = []
        for X_sequence, target in train_data_loader:
            X_sequence, target = X_sequence.to(device), target.to(device)

            y_hat = model(X_sequence)
            train_loss = custom_loss_function(y_hat, target)

            val_batch_pred = torch.sum(torch.argmax(y_hat, dim=1) == target) / torch.tensor(target.shape[0])
            tra_acc_list.append(val_batch_pred.detach().cpu().numpy())

            train_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            epoch_loss.append(train_loss.item())
        loss_tra = np.mean(epoch_loss)

        # 验证集
        model.eval()
        epoch_loss = []
        val_acc_list = []
        for X_sequence, target in valid_data_loader:
            X_sequence, target = X_sequence.to(device), target.to(device)
            y_hat = model(X_sequence)
            valid_loss = custom_loss_function(y_hat, target)
            epoch_loss.append(valid_loss.item())

            val_batch_pred = torch.sum(torch.argmax(y_hat, dim=1) == target) / torch.tensor(target.shape[0])
            val_acc_list.append(val_batch_pred.detach().cpu().numpy())


        loss_val = np.mean(epoch_loss)
        if loss_val < val_best:
            val_best = loss_val
            acc_best = np.mean(val_acc_list)
            stop_count = 0
            best_model = deepcopy(model)
            print("Epoch: {}/{}".format(epoch, n_epochs))
            print('loss_tra',loss_tra)
            print('acc train',np.mean(tra_acc_list))
            print('loss_val',loss_val)
            print('acc val',np.mean(val_acc_list))
        else:
            stop_count += 1
        if stop_count == 30:
            break
    
    print('val_best',acc_best)
    torch.save(best_model.state_dict(), f'dnn_fold_{fold}_balanced')

    ###      use      fold: 0
################ Feature info: ###############
Node's feature shape:torch.Size([3655452, 300])
# model parameters: 3421463
Epoch: 0/1000
loss_tra 2.6028304265893025
acc train 0.26846093
loss_val 2.334945412782522
acc val 0.36126423
Epoch: 1/1000
loss_tra 2.3465975637021272
acc train 0.36054435
loss_val 2.2191563844680786
acc val 0.39478192
Epoch: 2/1000
loss_tra 2.2715879492137745
acc train 0.38586035
loss_val 2.146085739135742
acc val 0.41429976
Epoch: 3/1000
loss_tra 2.2163031671358193
acc train 0.40666592
loss_val 2.091152686339158
acc val 0.43063927
Epoch: 4/1000
loss_tra 2.183532497157221
acc train 0.41815066
loss_val 2.073611772977389
acc val 0.43656415
Epoch: 5/1000
loss_tra 2.1333184802013894
acc train 0.43409935
loss_val 2.033166087590731
acc val 0.44918826
Epoch: 6/1000
loss_tra 2.1162938128346984
acc train 0.43889803
loss_val 2.0299762670810404
acc val 0.45150545
Epoch: 7/1000
loss_tra 2.112400432254957
acc train 0.4400709
loss_val 2.02009286330

Epoch: 20/1000
loss_tra 2.0137831138527913
acc train 0.47029418
loss_val 1.9813633469434886
acc val 0.46666923
Epoch: 21/1000
loss_tra 2.000029044565947
acc train 0.47458845
loss_val 1.9764696863981395
acc val 0.46800324
Epoch: 25/1000
loss_tra 1.9928881318672844
acc train 0.4770874
loss_val 1.9740115725077116
acc val 0.46968442
Epoch: 26/1000
loss_tra 1.9785069087277287
acc train 0.4810633
loss_val 1.9696663618087769
acc val 0.47032458
Epoch: 31/1000
loss_tra 1.962324627586033
acc train 0.48591232
loss_val 1.9680878153214088
acc val 0.4717303
Epoch: 32/1000
loss_tra 1.964816515342049
acc train 0.4853598
loss_val 1.9675668386312632
acc val 0.47113153
Epoch: 35/1000
loss_tra 1.9609842953474625
acc train 0.486428
loss_val 1.9670148812807524
acc val 0.47173113
Epoch: 36/1000
loss_tra 1.9483164890952733
acc train 0.48991573
loss_val 1.9615417076991155
acc val 0.47380957
Epoch: 45/1000
loss_tra 1.9371356114097265
acc train 0.4935294
loss_val 1.9593458496607268
acc val 0.47469103
Epoch: 51/1

Epoch: 50/1000
loss_tra 1.9267287549765213
acc train 0.49680573
loss_val 1.9519365200629601
acc val 0.47729352
Epoch: 61/1000
loss_tra 1.8992202318232991
acc train 0.5052583
loss_val 1.951660523047814
acc val 0.47784764
Epoch: 76/1000
loss_tra 1.8776383363682292
acc train 0.51147896
loss_val 1.9512204069357653
acc val 0.4782789
Epoch: 81/1000
loss_tra 1.8703834787659024
acc train 0.5134174
loss_val 1.951187761930319
acc val 0.47858843
Epoch: 83/1000
loss_tra 1.881861708993497
acc train 0.510037
loss_val 1.9504552162610567
acc val 0.47864723
Epoch: 91/1000
loss_tra 1.8583763625310814
acc train 0.5170648
loss_val 1.94994279054495
acc val 0.4800654
Epoch: 100/1000
loss_tra 1.8574918669203053
acc train 0.5171831
loss_val 1.9481355410355787
acc val 0.479256
Epoch: 121/1000
loss_tra 1.8315741632295692
acc train 0.52505213
loss_val 1.9475786135746882
acc val 0.480146
val_best 0.480146
    ###      use      fold: 5
################ Feature info: ###############
Node's feature shape:torch.Size(

Epoch: 3/1000
loss_tra 2.2126709658166637
acc train 0.40912223
loss_val 2.0851462254157433
acc val 0.4325423
Epoch: 4/1000
loss_tra 2.182872584591741
acc train 0.41880324
loss_val 2.0824546813964844
acc val 0.4364199
Epoch: 5/1000
loss_tra 2.1318112280057826
acc train 0.43465576
loss_val 2.0387033315805287
acc val 0.44683436
Epoch: 6/1000
loss_tra 2.115119070592134
acc train 0.4398281
loss_val 2.031212416978983
acc val 0.44936034
Epoch: 7/1000
loss_tra 2.1125762721766597
acc train 0.44066706
loss_val 2.0287905748073873
acc val 0.45017797
Epoch: 8/1000
loss_tra 2.112057887989542
acc train 0.44054157
loss_val 2.0245107183089623
acc val 0.45172817
Epoch: 10/1000
loss_tra 2.0722732429919035
acc train 0.45301926
loss_val 1.998678968502925
acc val 0.45887578
Epoch: 11/1000
loss_tra 2.0571269118267557
acc train 0.45753366
loss_val 1.9958227964547963
acc val 0.45999205
Epoch: 15/1000
loss_tra 2.0379762151966925
acc train 0.46293077
loss_val 1.9816371569266686
acc val 0.46527243
Epoch: 16/1000


Epoch: 50/1000
loss_tra 1.923234394322271
acc train 0.49783716
loss_val 1.961644681600424
acc val 0.47460625
Epoch: 55/1000
loss_tra 1.914259135723114
acc train 0.50077486
loss_val 1.9610547469212458
acc val 0.47509304
Epoch: 56/1000
loss_tra 1.904486524540445
acc train 0.5038348
loss_val 1.9605921231783354
acc val 0.47556347
Epoch: 70/1000
loss_tra 1.8935004819994388
acc train 0.5068628
loss_val 1.9592926043730516
acc val 0.47651464
Epoch: 100/1000
loss_tra 1.8560731359150098
acc train 0.51782584
loss_val 1.9586470539753253
acc val 0.4774434
Epoch: 106/1000
loss_tra 1.8406657410704572
acc train 0.5224136
loss_val 1.9570633539786706
acc val 0.47850126
Epoch: 128/1000
loss_tra 1.8338965193085048
acc train 0.5243157
loss_val 1.9569996916330779
acc val 0.4785511
Epoch: 132/1000
loss_tra 1.8218416405760725
acc train 0.52748066
loss_val 1.9565120752041156
acc val 0.4788385
Epoch: 136/1000
loss_tra 1.8169071109398551
acc train 0.5292233
loss_val 1.9554472749049847
acc val 0.4803539
Epoch: 16

In [19]:
print('val_best',acc_best)

val_best 0.48111153


In [ ]:
# best  0.48996183
class ISO_Node_NN(nn.Module):  

    def __init__(self): 
        super(ISO_Node_NN, self).__init__()        
        self.net = nn.Sequential(
            nn.Linear(300,2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(2048,1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024,512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512,256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 23),
        )
        

    def forward(self, x):  
        x = self.net(x)
        return x